Задание 1

In [ ]:
#Создаем соединение с SQL (+создаем ввод пароля)
import psycopg2
import getpass

conn = psycopg2.connect(user='postgres', 
                        database='postgres', 
                        host='localhost', 
                        port='5433', 
                        password = getpass.getpass('Insert password: ')
                       )
print(conn.get_dsn_parameters())

Задание 2

In [27]:
# Получение данных
#Нам необходимы следующие данные: ISIN, тип купона (постоянный и дисконтный), столбец, указывающий, является ли 
#облигация амортизационной, купон, количество купонов в год, цена и дата обратного выкупа по оферете,
#кколичество выплаченных купонов, номинальная стоимость, дата погашения облигации, дата выпуска облигации (как дата начала торгов)
#Из всего пречисленного нас особо интересуют непогашенные облигации, также убираем отсутствующие значения ISIN
#Поскольку по облигации может быть разное значение купона, отбираем только максимальное его значение, также берем 
#максимальное значение buybackdate и buybackprice для отбора только одного для каждой облигации
import pandas as pd
query = """SELECT "ISIN", "IssuerName", "CouponType", "AmortisedMty", "CouponPerYear", "NumCoupons",
            "CPN", "BUYBACKDATE", "BUYBACKPRICE", "FaceValue", "EndMtyDate", "CPN_PERIOD", "FirstDate", "HaveOffer"
            FROM (
				SELECT DISTINCT public.bond_description."ISIN, RegCode, NRDCode", public.bond_description."IssuerName", public.bond_description."CouponType", 
				public.bond_description."AmortisedMty", public.bond_description."CouponPerYear", public.bond_description."NumCoupons",
				public.bond_description."FaceValue", public.bond_description."EndMtyDate", "HaveOffer"
				FROM public.bond_description INNER JOIN public.quotes
				ON public.quotes."ISIN" = public.bond_description."ISIN, RegCode, NRDCode"  
				WHERE "ISIN, RegCode, NRDCode" IS NOT NULL AND "Status" != 'Погашен') AS S1
			INNER JOIN (SELECT "ISIN", MAX("CPN_PERIOD") AS "CPN_PERIOD", MAX("CPN") AS "CPN", MAX("BUYBACKDATE") AS "BUYBACKDATE", MAX("BUYBACKPRICE") AS "BUYBACKPRICE", MIN("TIME") AS "FirstDate" 
                FROM public.quotes GROUP BY "ISIN") AS S2
			ON S1."ISIN, RegCode, NRDCode"=S2."ISIN";"""
df = pd.read_sql_query(query, conn) 
df['EndMtyDate'] = pd.to_datetime(df['EndMtyDate'])
df.BUYBACKPRICE = df.BUYBACKPRICE.fillna(0) 
df

,ISIN,IssuerName,CouponType,AmortisedMty,CouponPerYear,NumCoupons,CPN,BUYBACKDATE,BUYBACKPRICE,FaceValue,EndMtyDate,CPN_PERIOD,FirstDate,HaveOffer
0,RU000A0JTR15,Ростелеком,Переменный,True,2,10,40.3900,9999-05-05,100.0,1000.00,2018-03-06,182.0,2013-03-22,True
1,RU000A0JULC6,РОСБАНК,Фиксированный,False,4,40,0.0300,None,0.0,1000.00,2024-04-30,92.0,2014-04-30,False
2,RU000A0JVCK6,Первое КоллекторБюро,Постоянный,True,4,12,42.3836,None,0.0,1000.00,2018-04-19,91.0,2015-04-23,False
3,RU000A0JURL4,ВТБ Лизинг Финанс,Переменный,True,4,40,34.2800,9999-05-05,100.0,1000.00,2024-07-17,91.0,2014-07-30,True
4,RU000A0JVPQ5,ПИК ГК,Переменный,False,2,20,71.0548,9999-05-05,100.0,1000.00,2025-08-08,182.0,2015-08-21,True
5,RU000A0JTW34,Вертолеты России,Переменный,False,2,10,41.1400,9999-05-05,100.0,1000.00,2018-04-20,182.0,2013-05-29,True
6,RU000A0JTU85,РЖД,Переменный,False,2,30,40.8900,9999-05-05,100.0,1000.00,2028-03-21,182.0,2013-04-12,True
7,RU000A0JUSH0,ЮниКредит Банк,Переменный,False,2,10,52.3600,9999-05-05,100.0,1000.00,2019-08-06,182.0,2014-08-13,True
8,RU000A0JU146,Восточный Экспресс КБ,Переменный,False,2,10,92.2500,9999-05-05,100.0,1000.00,2018-07-17,182.0,2013-07-23,True
9,RU000A0JUW31,Россельхозбанк,Переменный,False,4,40,27.6740,9999-05-05,100.0,1000.00,2024-09-27,91.0,2014-10-10,True


Задание 3

In [4]:
#Используем следуюшие аргументы: ISIN, дата, логин
def my_fun(ISIN, RegCode, NRDCode, date, db_login):
    conn=psycopg2.connect(user=db_login['user'],
                      database=db_login['database'],
                      host=db_login['host'],
                      port=db_login['port'],
                      password=getpass.getpass('Insert password: '))

In [50]:
#Здесь приведена попытка написать код для неамортизационных облигаций с постоянным купоном. 
#подозреваю, что нужно использовать еще pivot для вывода таблицы


import pandas as pd
import numpy as np
import datetime

df['EndMtyDate'] = pd.to_datetime(df['EndMtyDate'])
ISIN = df['ISIN']
coup = df['CPN']
coup_per_year = df['CouponPerYear']
coup_num = df['NumCoupons']
df['FirstDate'] = pd.to_datetime(df['FirstDate'])
buyback_date = df['BUYBACKDATE']
date = df['FirstDate']
buyback_price = df['BUYBACKPRICE']
date = pd.to_datetime(date)
AmortisedMty = df.AmortisedMty.astype(str)
CouponType = df.CouponType.astype(str)
payment = coup
df.CPN_PERIOD = pd.to_timedelta(df.CPN_PERIOD)
for i in range (0,len(df)): 
    if df.AmortisedMty.values[i] == False and df.CouponType.values[i] == 'Постоянный':
        if df.BUYBACKDATE.values[i] is None:
            while date.values[i] < df.EndMtyDate.values[i]:
                payment.values[i] = coup.values[i]
                df.CPN_PERIOD = pd.to_timedelta(df.CPN_PERIOD)
                date.values[i] = date.values[i] + df.CPN_PERIOD.values[i]
                if date.values[i] == df.EndMtyDate.values[i]:
                    payment.values[i] = df.CPN.values[i] + df.FaceValue.values[i]
                    date.values[i] = df.EndMtyDate.values[i] 
        if df.BUYBACKDATE.values[i] is not None:
            while (date.values[i] < df.BUYBACKDATE.values[i]) and (df.BUYBACKDATE.values[i] < df.EndMtyDate.values[i]):
                payment.values[i] = df.CPN.values[i]
                date.values[i] = date.values[i] + df.CPN_PERIOD.values[i]
                if date.values[i] == df.BUYBACKDATE.values[i]:
                    payment.values[i] = df.CPN.values[i] + df.BUYBACKPRICE.values[i]
                    date.values[i] = df.BUYBACKDATE.values[i] 
    print('Step :'+str(i))

payment
# return(payment)
# return(payment)
#if df.AmortisedMty.values[i] == True and df.CouponType.values[i] == 'Постоянный':
      # if df.BUYBACKDATE.values[i] is None:
        
    
      # if df.BUYBACKDATE.values[i] is not None:    
            

#if df.CouponType.values[i] == 'Дисконт':
     # if df.BUYBACKDATE.values[i] is None:
        
    
     # if df.BUYBACKDATE.values[i] is not None:    

Step :0
Step :1
Step :2
Step :3
Step :4
Step :5
Step :6
Step :7
Step :8
Step :9
Step :10


KeyboardInterrupt: 